In [ ]:
# Import everything
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

plt.rcParams['figure.figsize'] = [8,5]
plt.rcParams['font.size'] =14
plt.rcParams['font.weight']= 'bold'
plt.style.use('seaborn-v0_8-whitegrid')

# creating a function to create adhusted R-Squared

def adj_r2(X, y, model):
    r2 = model.score(X, y)
    n = X.shape[0]
    p = X.shape[1]
    adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
    return adjusted_r2

In [ ]:
# Import dataset
df = pd.read_csv('./train.csv')
print('\nNumber of rows and columns in the data set: ', df.shape)
df.head()
# Drop useless columns like id
# df = df.drop(["id"], axis = 1)

# Basic information
df.describe()
df.info()

In [ ]:
# Check if dataset has empty, zero values or duplicates
# https://www.kaggle.com/discussions/getting-started/250322

plt.figure(figsize=(12,4))
sns.heatmap(df.isnull(),cbar=False,cmap='viridis',yticklabels=False)
plt.title('Missing value in the dataset')
# Empty entries by feature
df.isna().sum()

# Ways of handling missing data
# filling missing value using fillna()   
df.fillna(0) 
# filling a missing value with previous ones   
df.fillna(method ='ffill') 
# fill with next ones
df.fillna(method ='bfill') 
# filling a null values with string
df["Gender"].fillna("No Gender", inplace = True)  
# will replace Nan value in dataframe with value -99   
df.replace(to_replace = np.nan, value = -99)  

# to interpolate the missing values  
df.interpolate(method ='linear', limit_direction ='forward') 
df.interpolate(method='polynomial', order=5)
# interpolate for time series
df.interpolate('time')

# drop rows / columns if anything is missing   
df.dropna() # df.dropna(axis = 1) 
# drop rows only if all are missing    
df.dropna(how = 'all') 



# Check for duplicates
duplicates = df.duplicated()
df[duplicates]



# Calculate the count of zeroes per column
zero_count = (df == 0).sum()

# Calculate the percentage of zeroes per column
zero_percentage = (df == 0).mean() * 100

# Print the count and percentage of zeroes
print("Count of zeroes per column:\n", zero_count)
print("\nPercentage of zeroes per column:\n", zero_percentage)

In [ ]:
# Dealing with Outliers with IQR

# column_list = ["list of columns we wish to crop outliers"]
column_list = ["allelectrons_Total"]
df_outliers = pd.DataFrame(df.loc[:,column_list])

# Calculate IQR
Q1 = df_outliers.quantile(0.25)
Q3 = df_outliers.quantile(0.75)
IQR = Q3 - Q1

# Method 1. Set extreme values to some maximum/min
    # We can use IQR score to filter out the outliers by keeping only valid values
    # Replace every outlier on the upper side by the upper whisker
    print("Data which is outside 1.5 IQR:")

    whisker = Q3 + 1.5 * IQR
    for i, j in zip(np.where(df_outliers > whisker)[0], np.where(df_outliers > whisker)[1]):
        
        print(f"Row {i}  {column_list[j]}  {df_outliers.iloc[i,j]}")

        df_outliers.iloc[i,j] = whisker.iloc[j]
        
    # Replace every outlier on the lower side by the lower whisker
    whisker  = Q1 - 1.5 * IQR
    for i, j in zip(np.where(df_outliers < whisker)[0], np.where(df_outliers < whisker)[1]): 
        df_outliers.iloc[i,j] = whisker.iloc[j]
    
    # Combine back
    df_corrected = df.drop(column_list, axis = 1)
    df_corrected = pd.concat([df_outliers, df], axis = 1)

# Method 2: delete rows with any outliers
    row_list = ((df_outliers >= (Q1 - 1.5 * IQR)) & (df_outliers <= (Q3 + 1.5 * IQR))).any(axis=1)
    df_corrected = df_outliers[row_list]

# once sure then overwrite
df = df_corrected

In [ ]:
# Feature Engineering: adding new features
# https://www.kaggle.com/code/vinayakshanawad/random-forest-with-bootstrap-sampling-for-beginner 


In [ ]:
# Data Visualisation with various plots

X = df.drop('Hardness', axis = 1)
y = df['Hardness']
# Violin Plot / sns.boxplot
    # inner="quart" or "box"
    # hue="diagnosis" (categorical result variable we care about)

    # Just showing the histograms without indication of result variable
    plt.figure(figsize=(10,10))
    sns.violinplot(data = X)
    plt.xticks(rotation=90)
    plt.show()

    # For continous response variable
    # For a feature with categorical data (M/F, 0/1/2/3) , show them side by side for each category
    plt.figure(figsize=(10,10))
    sns.violinplot(x="feature", y="Hardness", data=df, split=True) 
    plt.xticks(rotation=90)
    plt.show()

    # For Yes/no response variable "diagnosis", show two histograms side by side
    # This shows if this feature would be a good predictor, if the two distributions are very different
    X_norm = (X - X.mean()) / (X.std()) 
    data = pd.concat([y,X_norm],axis=1)
    data = pd.melt(data,id_vars="diagnosis",
                        var_name="features",
                        value_name='value')
    plt.figure(figsize=(10,10))
    sns.violinplot(x="features", y="value", hue="diagnosis", data=data, split=True)
    plt.xticks(rotation=90)
    plt.show()


# Joint plot
    # Essentially linear regression + histogram of two features with pearson's R correlation statistic
    sns.jointplot(x = X.loc[:,'feature1'], y = X.loc[:,'feature2'], kind="reg", color="#ce1414")

# Pair plot
    # Scatter plot/histogram of pairwise features
    sns.pairplot(X, diag_kind = 'kde', kind = "reg", corner = True) # or use df

# Scatter plot against response
    plt.figure(figsize = (20, 15))
    plotnumber = 1

    for column in df:
        if plotnumber <= 14:
            ax = plt.subplot(3, 5, plotnumber)
            sns.scatterplot(x = df[column], y = df['Hardness'])
            
        plotnumber += 1

    plt.tight_layout()
    plt.show()




In [ ]:
# Feature Selection on Numerical Columns

# X = X.loc[:,column_list] # pick out the numerical columns
X = (X - X.mean()) / (X.std()) 

# correlation plot
    corr = df.corr()
    sns.heatmap(corr, cmap = 'Wistia', annot= True)
    # Look by eye which rows look the same

# checking for multicollinearity using VIF
    from statsmodels.stats.outliers_influence import variance_inflation_factor
    vif = pd.DataFrame()
    vif['VIF'] = [variance_inflation_factor(X, i) for i in range(X.shape[1])]
    vif['Features'] = X.columns
    vif
    # VIF = 1/(1-r^2) where r^2 is the result of regressing this feature onto the remaining ones
    # VIF = 1: no correlation
    # VIF > 10: high correlation, consider dropping it
    # drop column and re-run

# PCA
    from sklearn.decomposition import PCA
    # PCA gives an idea of how many features are relevant, but best not to use those combinations since it is hard to explain
    # No need to split train and test if just getting an idea
    pca = PCA()
    pca.fit(X)
    component_names = [f"PC{i+1}" for i in range(X.shape[1])]
    X_pca = pd.DataFrame(pca.components_, columns=component_names, index=X.columns)
    X_pca.head()
    # If we think this is a reasonable combination of stuff, we should pca.transform(X)

    plt.figure(1, figsize=(14, 13))
    plt.clf()
    plt.axes([.2, .2, .7, .7])
    plt.plot(pca.explained_variance_ratio_, linewidth=2)
    plt.axis('tight')
    plt.xlabel('n_components')
    plt.ylabel('explained_variance_ratio_')



# several methods of feature selection
# https://www.kaggle.com/code/kanncaa1/feature-selection-and-data-visualization

# sklearn Select K best features
    from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
    # f_regression computes the F-test ANOVA score for a linear regression between feature and target
    # mutual_info_regression computes score between features
    # for classification task, use f_classif, mutual_info_classif

    select_feature = SelectKBest(f_regression, k=5).fit(X, y)
    feature_scores = pd.DataFrame({"Features": X.columns,
                "Score": select_feature.scores_
                })
    print(feature_scores)

    # Reduce x_train to the k best features
    X = select_feature.transform(X)

# See Random Forest and XGBoost as well


In [ ]:
# Normalising data for regression and splitting into train and test
# https://www.kaggle.com/code/humvale/sklearn-numerical-and-categorical-pipeline
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

# Separating data and target
X = df.drop(columns = ['Hardness'], axis = 1) # and drop any other columns from previous feature selection
y = df['Hardness']

# if the target y is categorical, then replace it with 0 ~ n-1. If not then comment out the below
# But DO NOT APPLY REGRESSION to this, and separate it out from the selector below
y_preprocessor = LabelEncoder()
y_preprocessor.fit_transform(y)

# Now deal with X
numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(df)
categorical_columns = categorical_columns_selector(df)

categorical_preprocessor = OneHotEncoder(handle_unknown="ignore") # if it sees a new category, it gets all 0s without being in a new category
numerical_preprocessor = StandardScaler()

preprocessor = ColumnTransformer([
    ('one-hot-encoder', categorical_preprocessor, categorical_columns),
    ('standard-scaler', numerical_preprocessor, numerical_columns)])

# split data train 70 % and test 30 %
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

x_train = preprocessor.fit_transform(x_train)
x_test = preprocessor.transform(x_test)
x_train = pd.DataFrame(x_train, columns=X.columns)
x_test = pd.DataFrame(x_test, columns=X.columns)

# If y is numerical then transform it also
y_train = preprocessor.fit_transform(y_train)
y_test = preprocessor.transform(y_test)
y_train = pd.DataFrame(y_train, columns=y.columns)
y_test = pd.DataFrame(y_test, columns=y.columns)

In [ ]:
# Random Forest

# Use this to:
# 1. estimate how many features are important
# 2. This is also a predictor

# Inputs
# n_estimators: numbers of trees, more is better but slower
# bootstrap: True
# max_samples: The fraction of samples to draw from X to train each tree [0,1]
# Bagging fraction = (1-1/N)^N'=e^(-max_samples)
# max_features: proportion of total features to use, try sqrt or log2
# max_depth: too much of this causes over fitting
# min_samples_split: causes underfitting
# min_samples_leaf: some value > 1 is slightly better, after that underfits

# Classification Task
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import confusion_matrix, accuracy_score

    rf = RandomForestClassifier(n_estimators = 100, random_state=43, max_features="log2", min_samples_leaf=3,max_depth=5)      
    rf.fit(x_train,y_train)

    ac = accuracy_score(y_test,rf.predict(x_test))
    print('Accuracy is: ',ac)
    cm = confusion_matrix(y_test,rf.predict(x_test))
    sns.heatmap(cm,annot=True,fmt="d")

# Continous Output: RF Regressor
    from sklearn.ensemble import RandomForestRegressor
    rf = RandomForestRegressor(n_estimators = 100, random_state = 42, max_features="log2", min_samples_leaf=3,max_depth=5)
    rf.fit(x_train, y_train)
    print(rf.score(x_train, y_train))
    print(rf.score(x_test, y_test))


In [ ]:
# K Nearest Neighbours
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
# https://www.kaggle.com/code/mjamilmoughal/k-nearest-neighbor-classifier-to-predict-fruits
# use classifier / regressor
# n_neighbors: vary this to find optimal value
# p uses Lp norm as distance
scores = pd.DataFrame(columns=["test_score"])
for k in range(5, 40):
    knn = KNeighborsRegressor(n_neighbors=k, weights = 'distance',p = 2)
    knn.fit(x_train,y_train)
    scores.loc[k,'test_score'] = knn.score(x_test,y_test)

scores.plot.line()

In [ ]:
# Linear Regression OLS (Detailed)

# With detailed report
import statsmodels.formula.api as smf
# formula = 'y ~ x' as column names
lm = smf.ols(formula = 'MEDV ~ RAD', data = df).fit()
lm.summary()

# Basic version
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(x_train, y_train)
lr.score(x_train, y_train)
lr.score(x_test, y_test)
print(adj_r2(x_train, y_train, lr))
print(adj_r2(x_test, y_test, lr))
y_pred = lr.predict(x_test)
coefficients = pd.DataFrame(lr.coef_,index = x_train.columns,columns=["coef"])
coefficients.sort_values(by="coef", ascending=False).plot.bar()

In [ ]:
# Logistic Regression
# https://www.kaggle.com/code/prashant111/logistic-regression-classifier-tutorial

In [ ]:
# LASSO, Ridge, Elastic Net with CV
# Description: https://www.kaggle.com/code/shweta2407/how-to-choose-best-regression-model 
# Detailed alpha search: https://www.kaggle.com/code/fugacity/ridge-lasso-elasticnet-regressions-explained
# LASSO
    from sklearn.linear_model import Lasso, LassoCV

    lasso_cv = LassoCV(alphas = None, cv = 10, max_iter = 100000, normalize = True)
    lasso_cv.fit(x_train, y_train)

    # best alpha parameter
    alpha = lasso_cv.alpha_
    lasso = Lasso(alpha = lasso_cv.alpha_)
    lasso.fit(x_train, y_train)
    lasso.score(x_train, y_train)
    lasso.score(x_test, y_test)
    print(adj_r2(x_train, y_train, lasso))
    print(adj_r2(x_test, y_test, lasso))

    # To investigate how the fitting happened, use LARS
    from sklearn.linear_model import lars_path
    alphas, _, coefs = lars_path(x_train, y_train, method='lasso', verbose=True)

    xx = np.sum(np.abs(coefs.T), axis=1)
    xx /= xx[-1]

    plt.plot(xx, coefs.T)
    ymin, ymax = plt.ylim()
    plt.vlines(xx, ymin, ymax, linestyle='dashed')
    plt.xlabel('|coef| / max|coef|')
    plt.ylabel('Coefficients')
    plt.title('LASSO Path')
    plt.axis('tight')
    plt.show()

# Ridge Regression
    from sklearn.linear_model import Ridge, RidgeCV

    alphas = np.random.uniform(0, 10, 50)
    ridge_cv = RidgeCV(alphas = alphas, cv = 10, normalize = True)
    ridge_cv.fit(x_train, y_train)
    # best alpha parameter
    alpha = ridge_cv.alpha_
    ridge = Ridge(alpha = ridge_cv.alpha_)
    ridge.fit(x_train, y_train)
    ridge.score(x_train, y_train)
    ridge.score(x_test, y_test)
    print(adj_r2(x_train, y_train, ridge))
    print(adj_r2(x_test, y_test, ridge))

    # to see the regularization path:
    from sklearn.linear_model import LassoLarsCV
    lassolarscv = LassoLarsCV().fit(x_train, y_train)
    coef_path = pd.DataFrame(lassolarscv.coef_path_,index=x_train.columns).T
    coef_path["coef_l2"] = np.sqrt(np.square(coef_path).sum(axis=1))
    coef_max = coef_path["coef_l2"].max()
    coef_path["coef_ratio"] = coef_path["coef_l2"]/coef_max
    plt.plot(coef_path["coef_ratio"], coef_path.iloc[:,0:7])
    plt.show()

# Elastic Net
    from sklearn.linear_model import ElasticNet, ElasticNetCV

    elastic_net_cv = ElasticNetCV(alphas = None, cv = 10, max_iter = 100000, normalize = True)
    elastic_net_cv.fit(x_train, y_train)

    alpha = elastic_net_cv.alpha_
    # l1 ratio 
    elastic_net_cv.l1_ratio

    elastic_net = ElasticNet(alpha = elastic_net_cv.alpha_, l1_ratio = elastic_net_cv.l1_ratio)
    elastic_net.fit(x_train, y_train)
    elastic_net.score(x_train, y_train)
    elastic_net.score(x_test, y_test)
    print(adj_r2(x_train, y_train, elastic_net))
    print(adj_r2(x_test, y_test, elastic_net))

# Show weights
    import eli5
    for model in [lasso, ridge, elastic_net]:
        print(f"model: {model}")
        eli5.show_weights(model, top=-1, feature_names = x_train.columns.tolist())
    

In [ ]:
# Kernel Ridge
# https://www.kaggle.com/code/kzhoulatte/kernel-ridge-regression-lb0-0571-rbf-laplacian 
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import polynomial_kernel, rbf_kernel, laplacian_kernel

kernel_list = ['linear', 'polynomial', 'rbf', 'laplacian']
for kernel in kernel_list:
    model = KernelRidge(kernel ='linear', alpha=1.0)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    rmse = np.sqrt(mean_squared_error(y_test,y_pred))
    print(f"Kernel {kernel} RMSE {rmse}")

In [ ]:
# Robust regression: Theil-sen, RANSAC, Huber
# These are better at dealing with outliers
# https://www.kaggle.com/code/plarmuseau/better-regression-techniques 
from sklearn.linear_model import LinearRegression, RANSACRegressor, TheilSenRegressor, HuberRegressor

estimators = [('OLS', LinearRegression()),
              ('Theil-Sen', TheilSenRegressor(random_state=42)),
              ('RANSAC', RANSACRegressor(random_state=42)),
              ('HuberRegressor', HuberRegressor())]
for name, estimator in estimators:
    estimator.fit(x_train, y_train)
    print(name,estimator.score(x_train, y_train))
    print(name,estimator.score(x_test, y_test))





In [ ]:
# XGBoost
# https://www.kaggle.com/code/stuarthallows/using-xgboost-with-scikit-learn
import xgboost as xgb
from scipy.stats import uniform, randint
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

# objective: "reg:linear"  "binary:logistic"  "multi:softprob"
xgb_model = xgb.XGBRegressor(objective="reg:linear", random_state=42)

xgb_model.fit(x_train, y_train)
y_pred = xgb_model.predict(x_test)
rmse=np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse}")
# Cross validation scores
scores = cross_val_score(xgb_model, X, y, scoring="neg_mean_squared_error", cv=5)
print(f"Scores: {np.sqrt(-scores)}")

# Hyperparameter Searching
# objective: "reg:linear"  "binary:logistic"  "multi:softprob"
xgb_model = xgb.XGBRegressor(objective="reg:linear", random_state=42)

params = {
    "n_estimators": randint(100, 150), # default 100
    "learning_rate": uniform(0.03, 0.3), # default 0.1 
    "max_depth": randint(2, 6), # default 3
    "subsample": uniform(0.6, 0.4),
    "colsample_bytree": uniform(0.7, 0.3),
    "gamma": uniform(0, 0.5),
    "alpha": uniform(0, 10),
    
}

search = RandomizedSearchCV(xgb_model, param_distributions=params, random_state=42, n_iter=200, cv=10, verbose=1, n_jobs=1, return_train_score=True)

search.fit(x_train, y_train)
results = search.cv_results_
candidates = np.flatnonzero(results['rank_test_score'] == 1)
for candidate in candidates:
    print("Best Models")
    print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
            results['mean_test_score'][candidate],
            results['std_test_score'][candidate]))
    print("Parameters: {0}".format(results['params'][candidate]))
    print("")

# Then use these parameters to define a newxgb_model again

# Early stopping
xgb_model.fit(x_train, y_train, early_stopping_rounds=10, eval_set=[(x_test, y_test)], verbose=False)
xgb.plot_importance(xgb_model)


In [ ]:
# XGBoost with Cross-Validation
# https://www.kaggle.com/code/prashant111/xgboost-k-fold-cv-feature-importance 
# This uses decision trees and adds them one at a time
# Using train-test split, this tells us if XGboost is a good model or not
# Once we are happy, we do CV on the entire dataset to get a slightly better model for future predictions

import xgboost as xgb
from xgboost import XGBRegressor, cv
from sklearn.metrics import accuracy_score

data_dmatrix = xgb.DMatrix(data=x_train,label=y_train)
# data_dmatrix = xgb.DMatrix(data=X,label=y)

params = {"objective":"binary:logistic", # "reg:squarederror"
          'colsample_bytree': 0.3,
          'learning_rate': 0.1,
          'max_depth': 5, 
          'alpha': 10}

xgb_cv = cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50, early_stopping_rounds=10, metrics="auc", as_pandas=True, seed=123)

my_model = XGBRegressor(n_estimators=1000)
# n_estimators specifies how many times to go through the modeling cycle
# verbose = print messages
my_model.fit(x_train, y_train, verbose=False )
print('XGBoost model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

In [ ]:
# As I understand, you first use xgb.cv (the cross validation function of XGB) to "tune" the parameters you will use in your model (mainly the optimal nrounds).
# Once you had found the optimal parameters for your dataset (that gave you the minimal logloss/error) you then replicate this parameters in a training function of the XGBoost package, that are xgboost or xgb.train to obtain the model you can use to predict on the test set.
    
# XGBoost Classifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# declare parameters
params = {
            'objective':'binary:logistic',
            'max_depth': 4,
            'alpha': 10,
            'learning_rate': 1.0,
            'n_estimators':100
        }         
           
# instantiate the classifier 
xgb_clf = XGBClassifier(**params)
# fit the classifier to the training data
xgb_clf.fit(x_train, y_train)
# we can view the parameters of the xgb trained model
print(xgb_clf)
# make predictions on test data
y_pred = xgb_clf.predict(x_test)
# compute and print accuracy score
print('XGBoost model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

# See the importance of various features
xgb.plot_importance(xgb_clf)
plt.figure(figsize = (16, 12))
plt.show()



# import XGBoost
import xgboost as xgb
# define data_dmatrix
from xgboost import cv

params = {"objective":"binary:logistic",'colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10}

xgb_cv = cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50, early_stopping_rounds=10, metrics="auc", as_pandas=True, seed=123)

In [ ]:
# ML
# https://www.kaggle.com/code/dimitreoliveira/deep-learning-for-time-series-forecasting